In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
# Constant Define
element_size = 28; time_steps = 28; num_classes = 10
batch_size = 128; hidden_layer_size = 128

# Placeholder
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')

### 내장함수를 사용한 RNN

In [7]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

### Linear Layer 

In [8]:
Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))

In [9]:
def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl

In [10]:
last_rnn_output = outputs[:, -1, :]
final_output = get_linear_layer(last_rnn_output)

### Loss Function / Train

In [12]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=y))
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

### Accuracy

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

### Sess.run()

In [13]:
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, element_size))
    sess.run(train_step, feed_dict={_inputs: batch_x, y: batch_y})

    if i == 0 or (i+1) % 1000 == 0:
        acc, loss, = sess.run([accuracy, cross_entropy], feed_dict={_inputs: batch_x, y: batch_y})
        print("Iter " + str(i+1) + ", Minibatch Loss= " +
              "{:.6f}".format(loss) + ", Training Accuracy= " +
              "{:.5f}".format(acc))
print("Complete")

Iter 1, Minibatch Loss= 2.305145, Training Accuracy= 7.81250
Iter 1000, Minibatch Loss= 0.125498, Training Accuracy= 97.65625
Iter 2000, Minibatch Loss= 0.090851, Training Accuracy= 98.43750
Complete
